In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI
import json
import asyncio
import argparse
from tqdm.asyncio import tqdm_asyncio
import re
from tqdm import tqdm
def get_response(text, system_setting="You are a helpful assistance."):
    client = OpenAI(
        api_key="", # 如果您没有配置环境变量，请在此处用您的API Key进行替换
        base_url="",  # 填写DashScope服务的base_url
    )
    # client = OpenAI(
    #     api_key="sk-HPns8rdhFIuha10rKMqZUCws8lxAnlHVI69QxMV5VI3UEj4w", # 如果您没有配置环境变量，请在此处用您的API Key进行替换
    #     base_url="https://api.moonshot.cn/v1",  # 填写DashScope服务的base_url
    # )
    response = client.chat.completions.create(
        model="qwen-max",
        messages=[{'role': 'system', 'content': system_setting},
                  {'role': 'user', 'content': text}],
        max_tokens=512,
        temperature=0.7,
        top_p=0.7,
        )
    return response.choices[0].message.content

def get_more_q_js(origin_q, prompt_path, retry_num=4):
    prompt=""
    with open(prompt_path,"r",encoding="utf-8") as f:
        prompt=f.read()
    input=prompt.replace("{text}", origin_q)
    i_str=get_response(input)
    i_js=[]
    flag=False
    for i in range(retry_num):
        try:
            # if not (i_str.endswith("]")):
            #     print("1")
            #     if (i_str.endswith(",")):
            #         i_str=i_str+"]"
            #     elif (i_str.endswith("}")):
            #         i_str=i_str+",]"
            #     else:
            #         i_str=i_str+"},]"
            
            i_js=json.loads(i_str[i_str.find("["):i_str.find("]")+1])

            if not all(isinstance(item, str) for item in i_js):
                raise ValueError("get_i_js过程中得到的list里面并不是都装着str对象")
            flag=True
        except:
            i_str=get_response(input)
    if(flag==False):
        print(f"生成更多指令的结果格式错误：{i_str}")
        i_str=""
        i_js=[]
    i_js.insert(0, origin_q)
    return i_js

def add_prefix(new_instructions, instruction_prefix):
    for i in range(len(new_instructions)):
        try:
            new_instructions[i]=instruction_prefix+new_instructions[i]
        except:
            print(f"wrong new_instruction:{new_instructions}")
            print(f"wrong new_instruction[{i}]:{new_instructions[i]}")
    return new_instructions

def adapt_to_training_format(ori_js, output_path):
    rs=[]
    for i in range(len(ori_js)):
        insturctions=ori_js[i]["instruction"]
        input=ori_js[i]["input"]
        output=ori_js[i]["output"]
        temp=[]
        for j in range(len(insturctions)):
            temp.append({"instruction":insturctions[j], "input":input, "output":output})
        rs.extend(temp)

    with open(output_path, "w", encoding="utf-8") as f: 
        f.write(json.dumps(rs,ensure_ascii=False, indent=4))
    return   

if __name__=='__main__':
    m_q_prompt_path="./get_more_q_prompt1.txt"
    instruction_prefix="你是一个精通认知行为心理疗法的专家。请根据输入的文本回答以下问题或完成以下任务："
    input_path="./案例9/案例分析与评价/output_案例9_案例分析与评价.txt"
    output_path="./案例9/案例分析与评价/final_output_案例9_案例分析与评价.txt"
    # input_path="./案例9/背景+个案概念化/output_案例9_案例描述+概念化.txt"
    # output_path="./案例9/背景+个案概念化/final_output_案例9_案例描述+概念化.txt"
    # input_path="./案例9/治疗方案/output_案例9_治疗方案.txt"
    # output_path="./案例9/治疗方案/final_output_案例9_治疗方案.txt"

    json_ori_result=0
    with open(input_path,"r",encoding="utf-8") as f:
        s=f.read()
        try:
            json_ori_result=json.loads(s)
        except:
            print(f"初步结果json格式错误。路径{m_q_prompt_path}")
    json_final_rs=json_ori_result
    
    for i,single_rs in tqdm(enumerate(json_final_rs), total=len(json_final_rs)):
        
        new_instructions=get_more_q_js(single_rs["instruction"], m_q_prompt_path)
        
        new_instructions=add_prefix(new_instructions, instruction_prefix)
        #print(new_instructions)
        json_final_rs[i]["instruction"]=new_instructions

    adapt_to_training_format(json_final_rs, output_path)

100%|██████████| 1/1 [00:07<00:00,  7.57s/it]


In [25]:
#print(json.dumps(json_final_rs,ensure_ascii=False, indent=4))